In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [23]:
df = pd.read_csv('dataset.csv') # load dataset
df=df.dropna() #dropping rows with missing values

In [24]:
# Since the scope of our project is smaller than similar projects, we will be setting limits
max_words = 5000
max_length = 100

tokenizer = Tokenizer(num_words=max_words)
# With this, we are trying to update the vocabulary based on the word frequency
tokenizer.fit_on_texts(df['title'])
# Since neural networks require inputs of the same shape, we need to make sure we have padding
sequences = tokenizer.texts_to_sequences(df['title'])
X = pad_sequences(sequences, maxlen=max_length)
y = to_categorical(df['label'])

In [25]:
# Here, we're going to split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# This is where we're trying to build the model
model = Sequential()
model.add(Embedding(max_words, 50, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# Softmax is a mathematical function that takes a vector of real numbers as input and transforms it into a probability  distribution
# Reference - https://deepai.org/machine-learning-glossary-and-terms/softmax-layer
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max', restore_best_weights=True)

In [27]:


# This is where we're trying to train the model
batch_size = 32
epochs = 15
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stopping])

# Here, we try to evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/15
18/18 [==============================] - 3s 54ms/step - loss: 0.6802 - accuracy: 0.5967 - val_loss: 0.6692 - val_accuracy: 0.6066
Epoch 2/15
18/18 [==============================] - 1s 39ms/step - loss: 0.5672 - accuracy: 0.7372 - val_loss: 0.6727 - val_accuracy: 0.6557
Epoch 3/15
18/18 [==============================] - 1s 39ms/step - loss: 0.4267 - accuracy: 0.8102 - val_loss: 0.5488 - val_accuracy: 0.7541
Epoch 4/15
18/18 [==============================] - 1s 39ms/step - loss: 0.2713 - accuracy: 0.8960 - val_loss: 0.4198 - val_accuracy: 0.8197
Epoch 5/15
18/18 [==============================] - 1s 40ms/step - loss: 0.1325 - accuracy: 0.9580 - val_loss: 0.3475 - val_accuracy: 0.8197
Epoch 6/15
18/18 [==============================] - 1s 40ms/step - loss: 0.0742 - accuracy: 0.9781 - val_loss: 0.2707 - val_accuracy: 0.8525
Epoch 7/15
18/18 [==============================] - 1s 39ms/step - loss: 0.0636 - accuracy: 0.9872 - val_loss: 0.3165 - val_accuracy: 0.8033
Epoch 8/15
18

In [28]:
# Make predictions on new data
new_data = ["The news headline (fake or not)"]
new_data_sequences = tokenizer.texts_to_sequences(new_data)
new_data_padded = pad_sequences(new_data_sequences, maxlen=max_length)
predictions = model.predict(new_data_padded)

# This is to print the predicted class probabilities
print(f'Predicted Probabilities: {predictions}')

# This is to convert predicted probabilities to class labels (fake or true)
predicted_classes = [1 if prob[1] > prob[0] else 0 for prob in predictions]
print(f'Predicted Classes: {predicted_classes}')

1/1 [==============================] - 0s 183ms/step
Predicted Probabilities: [[0.8778616  0.12213831]]
Predicted Classes: [0]
